<br></br><br></br><br></br><br></br><br></br><br></br><br></br><br></br>
## <center>Artificial Intelligence: Knowledge Representation and Planning</center>
### <center>CM0472: Assignment 3 – Clustering</center>
<br></br><br></br><br></br><br></br><br></br><br></br><br></br><br></br>
###### JABER RAHIMIFARD [875545]
###### AREF ENAYATI [882341]
<br></br><br></br><br></br><br></br>

### Assignment

Perform classification of the  Semeion Handwritten Digit data set using:

    -Latent class analysis;
    -Mean shift;
    -Normalized cut.
    
__Mean shift__ and __Normalized Cut__ assume that the images are vectors in a 256 dimensional Euclidean space.



Provide the code and the extracted clusters as the number of clusters k varies __from 5 to 15__, for __LCA and normalized-cut__, while for __Mean shift__ vary the kernel width. For each value of k (or kernel width) provide the value of the Rand index:<br></br><br></br>
__<center>R=2(a+b)/(n(n-1))</center>__

where
    <br>n is the number of images in the dataset.</br>
    <br>a is the number of pairs of images that represent the same digit and that are clustered together.</br>
    <br> is the number of pairs of images that represent different digits and that are placed in different clusters.</br>
Explain the differences between the three models.

__Tip:__ Bernoulli models can be visualized as a greyscale image to inspect the learned model.

<br></br><br></br><br></br><br></br>

### Overview

In this assignment we perform classification using 3 algorithms __LCA__, __Mean Shift__, and __Normalized Cut__ on the Semeion Handwritten Digit dataset.
<br>We modify the number of cluster in each cluster and the widthBand in case using mean shift algorithm.</br>


In [81]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.metrics.cluster import adjusted_rand_score, rand_score
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from scipy.special import logsumexp
from time import time

# import warnings filter
from warnings import simplefilter
# ignore all future warnings

In [82]:
def get_data():
    data = np.loadtxt('./semeion.data', dtype=np.int8)
    return data[:, :256], data[:, 256:]


def one_hot_decode(y: np.array):
    return np.argmax(y, axis=1)


def get_data_transformed():
    data = np.loadtxt('./semeion.data', dtype=np.int8)
    return data[:, :256], one_hot_decode(data[:, 256:])


In [83]:
# fake class for no dim. red.
class NoDR:
    def __init__(self, n_components):
        pass

    def fit_transform(self, X):
        return X.copy()


In [84]:
simplefilter(action='ignore', category=FutureWarning)
# vector containing the number of clusters
ks = range(5, 16)
# vector containing the dimensions for dimensionality reduction
ds = (2, 64, 128, 256)


In [85]:
def _log_no_underflow(x):
    """Computes log without underflow
    Parameters
    ----------
    x : float or numpy.ndarray
    Returns
    -------
    log(x) if x > 0 else log eps
    """
    return np.log(np.clip(x, 1e-15, 1))

In [86]:
def _estimate_bernoulli_parameters(X, resp):
    """Estimate the Bernoulli distribution parameters.
    Parameters
    ----------
    X : array-like, shape (n_samples, n_features)
        The input data array.
    resp : array-like, shape (n_samples, n_components)
        The responsibilities for each data sample in X.
    Returns
    -------
    nk : array-like, shape (n_components,)
        The numbers of data samples in the current components.
    means : array-like, shape (n_components, n_features)
        The centers of the current components.
    """
    nk = resp.sum(axis=0) + 10 * np.finfo(resp.dtype).eps
    means = np.dot(resp.T, X) / nk[:, np.newaxis]
    return nk, means

In [87]:
def _estimate_log_bernoulli_prob(X, means):
    """Estimate the log Bernoulli probability.
    Parameters
    ----------
    X : array-like, shape (n_samples, n_features)
    means : array-like, shape (n_components, n_features)
    Returns
    -------
    log_prob : array, shape (n_samples, n_components)
    """
    return (X.dot(_log_no_underflow(means).T) +
            (1 - X).dot(_log_no_underflow(1 - means).T))


In [88]:
class Bernoulli_Mixture:
    def __init__(self, n_components, max_iter=500, tol=1e-3):
        '''Mixture of Bernulli Model
        pi : mixture weights / mixing coefficients
        mu : Bernoulli parameters / means
        gamma : responsibilities / expected values z_nk, sample n comes from component k
        '''
        self.n_components = n_components
        self.max_iter = max_iter
        self.tol = tol

    def __str__(self):
        return 'BMM'

    def fit(self, X, init_kmeans=True):
        """Estimate model parameters using X and predict the labels for X.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            List of n_features-dimensional data points. Each row
            corresponds to a single data point.
        Returns
        -------
        labels : array, shape (n_samples,)
            Component labels.
        """

        self.converged_ = False

        n_samples, _ = X.shape
        random_state = 42
        if init_kmeans:
            resp = np.zeros((n_samples, self.n_components))
            label = (
                KMeans(n_clusters=self.n_components, n_init=1, random_state=random_state)
                    .fit(X)
                    .labels_
            )
            resp[np.arange(n_samples), label] = 1
        else:
            resp = random_state.rand(n_samples, self.n_components)
            resp /= resp.sum(axis=1)[:, np.newaxis]

        self._initialize(X, resp)

        lower_bound = -np.inf

        for n_iter in range(1, self.max_iter + 1):
            prev_lower_bound = lower_bound

            log_prob_norm, log_resp = self._e_step(X)
            self._m_step(X, log_resp)
            lower_bound = log_prob_norm

            change = lower_bound - prev_lower_bound

            if abs(change) < self.tol:
                self.converged_ = True
                break

        if not self.converged_:
            raise ValueError('Not converged')

        _, log_resp = self._e_step(X)

        self.labels_ = log_resp.argmax(axis=1)

    def _e_step(self, X):
        """E step.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
        Returns
        -------
        log_prob_norm : float
            Mean of the logarithms of the probabilities of each sample in X
        log_responsibility : array, shape (n_samples, n_components)
            Logarithm of the posterior probabilities (or responsibilities) of
            the point of each sample in X.
        """
        log_prob_norm, log_resp = self._estimate_log_prob_resp(X)
        return np.mean(log_prob_norm), log_resp

    def _estimate_weighted_log_prob(self, X):
        """Estimate the weighted log-probabilities, log P(X | Z) + log weights.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
        Returns
        -------
        weighted_log_prob : array, shape (n_samples, n_component)
        """
        return self._estimate_log_prob(X) + self._estimate_log_weights()

    def _estimate_log_prob_resp(self, X):
        """Estimate log probabilities and responsibilities for each sample.
        Compute the log probabilities, weighted log probabilities per
        component and responsibilities for each sample in X with respect to
        the current state of the model.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
        Returns
        -------
        log_prob_norm : array, shape (n_samples,)
            log p(X)
        log_responsibilities : array, shape (n_samples, n_components)
            logarithm of the responsibilities
        """
        weighted_log_prob = self._estimate_weighted_log_prob(X)
        log_prob_norm = logsumexp(weighted_log_prob, axis=1)
        with np.errstate(under="ignore"):
            # ignore underflow
            log_resp = weighted_log_prob - log_prob_norm[:, np.newaxis]
        return log_prob_norm, log_resp

    def _initialize(self, X, resp):
        """Initialization of the Bernoulli mixture parameters.
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
        resp : array-like, shape (n_samples, n_components)
        """
        n_samples, n_dim = X.shape

        weights, means = _estimate_bernoulli_parameters(X, resp)
        weights /= n_samples

        self.weights_ = weights
        self.means_ = means

    def _m_step(self, X, log_resp):
        """M step.
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
        log_resp : array-like, shape (n_samples, n_components)
            Logarithm of the posterior probabilities (or responsibilities) of
            the point of each sample in X.
        """
        n_samples, _ = X.shape
        self.weights_, self.means_ = (
            _estimate_bernoulli_parameters(X, np.exp(log_resp))
        )
        self.weights_ /= n_samples

    def _estimate_log_prob(self, X):
        return _estimate_log_bernoulli_prob(X, self.means_)

    def _estimate_log_weights(self):
        return np.log(self.weights_)


In [89]:
def LCA(p=False):
    '''Results for parametric clustering models'''
    # results will be inserted into a DataFrame
    columns = ('clustering_model', 'dim_red_model', 'd', 'k', 'time', 'ARI', 'RI')
    rows = []

    # read data and split
    X, y = get_data_transformed()
    # loop clusteting model
    for k in ks:
        for d in ds:
            if d != 256:
                embedding_model = PCA
            else:
                embedding_model = NoDR
            start_time = time()
            embedding = embedding_model(n_components=d)  # DR model
            X_transformed = embedding.fit_transform(X)
            model = Bernoulli_Mixture(k)  # clustering model
            model.fit(X_transformed)
            elapsed = time() - start_time
            title = f'Results for {model} on {d}-{embedding_model.__name__} - k={k} ({elapsed:.02f}s)'
            print(title)
            # RI score
            ari, ri = print_rand(y, model.labels_)
            # plot results
            if p: plot2D(X_transformed, model.labels_, title)
            result = (str(model), embedding_model.__name__, d, k, elapsed, ari, ri)
            rows.append(result)
    # save results to file
    df = pd.DataFrame(data=rows, columns=columns)
    print(df)
    df.to_csv('LCA.csv', index=False)


In [90]:
def print_rand(y_true, y_pred):
    'Print and return the Rand Indexes'
    ari = adjusted_rand_score(y_true, y_pred)
    ri = rand_score(y_true, y_pred)
    print('adjusted_rand_score', round(ari * 100, 2), '%')
    print('rand_score', round(ri * 100, 2), '%', end='\n\n')
    return ari, ri

In [91]:
def results_analysis():
    df = pd.read_csv('LCA.csv')

    dfg_model = df.groupby('clustering_model').mean()[['time', 'ARI', 'RI']]
    print('averages by clustering model:')
    print(dfg_model.sort_values('ARI', ascending=False))
    print()

    dfg_k = df.groupby('k').mean()[['time', 'ARI', 'RI']]
    print('averages by k:')
    print(dfg_k.sort_values('ARI', ascending=False))
    print()

    dfg_dr = df.groupby('dim_red_model').mean()[['time', 'ARI', 'RI']]
    print('averages by DR model:')
    print(dfg_dr.sort_values('ARI', ascending=False))
    print()

    dfg_both = df.groupby(['clustering_model', 'dim_red_model']).mean()[['time', 'ARI', 'RI']]
    print('averages by clustering & DR model:')
    print(dfg_both.sort_values('ARI', ascending=False))
    print()

    print('best ARI achieved by:')
    print(df.sort_values('ARI', ascending=False).head(3))

In [92]:
def main():
    
    LCA()
    results_analysis()

In [93]:

if __name__ == '__main__':
    main()


Results for BMM on 2-PCA - k=5 (0.02s)
adjusted_rand_score 16.07 %
rand_score 72.39 %

Results for BMM on 64-PCA - k=5 (0.07s)
adjusted_rand_score 23.04 %
rand_score 79.41 %

Results for BMM on 128-PCA - k=5 (0.20s)
adjusted_rand_score 21.78 %
rand_score 79.42 %

Results for BMM on 256-NoDR - k=5 (0.09s)
adjusted_rand_score 25.26 %
rand_score 79.95 %

Results for BMM on 2-PCA - k=6 (0.04s)
adjusted_rand_score 17.13 %
rand_score 75.92 %

Results for BMM on 64-PCA - k=6 (0.08s)
adjusted_rand_score 27.1 %
rand_score 82.26 %

Results for BMM on 128-PCA - k=6 (0.15s)
adjusted_rand_score 22.84 %
rand_score 81.7 %

Results for BMM on 256-NoDR - k=6 (0.34s)
adjusted_rand_score 30.68 %
rand_score 83.59 %

Results for BMM on 2-PCA - k=7 (0.03s)
adjusted_rand_score 17.06 %
rand_score 76.77 %

Results for BMM on 64-PCA - k=7 (0.07s)
adjusted_rand_score 27.11 %
rand_score 84.03 %

Results for BMM on 128-PCA - k=7 (0.20s)
adjusted_rand_score 26.95 %
rand_score 84.24 %

Results for BMM on 256-NoDR - 